# Content-Based Recommender Systems and Association Rules

### Table of Contents
1. [Introduction](#introduction)
2. [Data Information](#data-info)
3. [Load Libraries](#load-libraries)
4. [Load Datasets](#load-datasets)
5. [Preprocessing](#preprocessing)
6. [Association Rules Mining](#association-rules-mining)

## Introduction <a class="anchor" id="introduction"></a>

Basic association analysis just deals with the occurrence of one item with another. More complicated analysis can take into consideration the quantity of occurrence, price, and sequence of occurrence, etc. The method for finding association rules through data mining involves the following sequential steps:
* **Step 1:** Prepare the data in transaction format. An association algorithm needs input data to be formatted in a particular format.
* **Step 2:** Short-list frequently occurring item sets. Item sets are combination of items. An association algorithm limits the analysis to the most frequently occurring items, so the final rule set extracted in next step is more meaningful.
* **Step 3:** Generate relevant association rules from item sets. Finally, the algorithm generates and filters the rules based on the interest measure.

One of the benefits of creating a recommender system with association rules it that its recommendations are highly interpretable. While methods like latent factoring methods are powerful, the trends from which their results are gleaned are cryptic. Association rules are simple: "The books you like frequently have these keywords? We'll recommend you more books like that."

While this may seem like a trivial point, remember our design principles! One of our goals was to have users trust that the recommendations we make will be relevant. Giving concrete explanations for each recommended item might give users more of an incentive to believe our recommendations.

## Data Information <a class="anchor" id="data-info"></a>

We will use one publically available dataset which will be used to explain the concepts: [Goodbooks-10k](https://www.kaggle.com/zygmunt/goodbooks-10k)

## Load Libraries <a class="anchor" id="load-libraries"></a>

Loading necessary libraries

In [1]:
import numpy as np 
import pandas as pd

from efficient_apriori import apriori

## Load Datasets <a class="anchor" id="load-datasets"></a>

Loading necessary datasets

In [2]:
# Import datasets
booksRaw =  pd.read_csv("../00_Dataset/goodbooks-10k/books.csv")
tagsRaw = pd.read_csv("../00_Dataset/goodbooks-10k/tags.csv")
bookTagsRaw = pd.read_csv("../00_Dataset/goodbooks-10k/book_tags.csv")
ratingsRaw = pd.read_csv("../00_Dataset/goodbooks-10k/ratings.csv")
to_readRaw = pd.read_csv("../00_Dataset/goodbooks-10k/to_read.csv")

# Merge bookTags and tags
all_tags = pd.merge(bookTagsRaw, tagsRaw, left_on='tag_id', right_on='tag_id', how='inner')
all_tags_grouped = all_tags.groupby("goodreads_book_id")

# Show first five rows of all_tags_grouped
display(all_tags.head())

,goodreads_book_id,tag_id,count,tag_name
0,1,30574,167697,to-read
1,2,30574,24549,to-read
2,3,30574,496107,to-read
3,5,30574,11909,to-read
4,6,30574,298,to-read


In [3]:
# Group ratings by user
ratingsGrouped = ratingsRaw.sort_values('user_id').groupby(by='user_id')

# Show rows of ratings by a particular user (e.g. 1)
ratingsGrouped.get_group(1)

,book_id,user_id,rating
117889,1180,1,4
488112,4893,1,3
625717,6285,1,4


## Preprocessing <a class="anchor" id="preprocessing"></a>

Then, make a user profile for each user, storing each rated item with a list of its associated keywords.

The next step is to resolve all conflicts, such as {sci-fi=like} and {sci-fi=dislike}. If we have both of these rules and they have similar confidence levels, we can’t reliably say that the “sci-fi” tag is associated with a like or a dislike.

In [4]:
class UserProfile:
  def __init__(self, user_id):
    self.userID = user_id;
    self.allTags = [];
    self.highWords = set()
    self.lowWords = set()
    
  def makeUserProfile(self):
      #Separate high rated books and low rated books
      ratedBooks = ratingsGrouped.get_group(self.userID)
      high = ratedBooks[ratedBooks.rating >= 3]
      low = ratedBooks[ratedBooks.rating < 3]
     
      for index, row in high.iterrows():
        #Add tags to list of keywords
        taglist = all_tags[all_tags.goodreads_book_id == row['book_id']].tag_name.tolist()
        
        #Add author names to list of keywords
        authors = booksRaw[booksRaw.book_id == row['book_id']].authors.tolist()
        if (authors != []):
          authors = authors[0].split(",")
          taglist.extend(authors)
            
        self.highWords.union(set(taglist))
        
        #Add rating keyword to transaction
        taglist.append("like")
        
        #Add keyword list to list of all transactions
        if (taglist != []):
          self.allTags.append(taglist)
          
      #repeat this process for the low rated books
      for index, row in low.iterrows():
        taglist = all_tags[all_tags.goodreads_book_id == row['book_id']].tag_name.tolist()
        
        authors = booksRaw[booksRaw.book_id == row['book_id']].authors.tolist()
        if (authors != []):
          authors = authors[0].split(",")
          taglist.extend(authors)
        
        self.lowWords.union(set(taglist))
        
        taglist.append("dislike")
        if (taglist != []):
          self.allTags.append(taglist)
      
      #Remove intersection of low and high keywords from all transactions 
      intersection = self.highWords.intersection(self.lowWords)
      for i in self.allTags:
        j = 0
        while j < len(i):
          if i[j] in intersection:
            i.pop(j)
          j += 1

In [5]:
user = UserProfile(17566)
user.makeUserProfile()

for i in range(len(user.allTags)):
    print(user.allTags[i])

['like']
['like']
['like']
['like']
['like']
['to-read', 'favorites', 'currently-reading', 'young-adult', 'fiction', 'books-i-own', 'owned', 'favourites', 'owned-books', 'all-time-favorites', 'default', 'my-books', 'i-own', 'audiobook', 'favorite-books', 'novels', 'favorite', 'audiobooks', 'my-library', 'english', 'british', 'read-in-2016', 'own-it', 'library', 'audio', 'novel', 'read-in-2015', 'shelfari-favorites', 'kindle', 'romance', 'to-buy', 'read-in-2014', 'ebook', 'contemporary', 'england', 'audio-books', 'read-in-2013', 'read-in-2011', 'borrowed', 'female-author', 'contemporary-fiction', 'wish-list', 'recommended', 'adult', 'book-club', 'adult-fiction', 'literature', 'abandoned', 'audible', 'ebooks', 'audio-book', 'did-not-finish', 'to-read-fiction', 'history', 'historical', 'didn-t-finish', 'australian', 'listened-to', 'bookclub', 'literary', 'read-in-2012', 'europe', 'britain', 'uk', 'want-to-read', 'survival', 'school', 'general-fiction', 'historical-fiction', 'literary-fict

## Association Rules Mining <a class="anchor" id="association-rules-mining"></a>


For each user profile, we mine all of its association rules for a given minimum support and minimum confidence.

The antecedent in content based system differs from collaborative filtering, where both the antecedent and consequent of rules are ratings. These rules will have some set of keywords in the left-hand side, and a consequent rating on the right.
Once we have all of these rules for each user, we can use them to start predict ratings.

In [6]:
print("Length of allTags: ", sum(len(tag) for tag in user.allTags))

Length of allTags:  2825


In [7]:
allItemsets, allRules = list(apriori(user.allTags,
                                     min_confidence=0.3,
                                     min_support=0.13))
print("Without minimum confidence and support, there would be over rules! Here we narrow it down to just", len(allRules),".")

Without minimum confidence and support, there would be over rules! Here we narrow it down to just 17896 .


In [8]:
for i in range(len(allRules)):
    print(allRules[i])

{currently-reading} -> {books-i-own} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own} -> {currently-reading} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{default} -> {books-i-own} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own} -> {default} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{favorites} -> {books-i-own} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own} -> {favorites} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{library} -> {books-i-own} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own} -> {library} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own} -> {like} (conf: 1.000, supp: 0.134, lift: 1.043, conv: 41237113.402)
{owned} -> {books-i-own} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own} -> {owned} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{owned-books} ->

{library, to-read} -> {books-i-own, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{library, owned-books} -> {books-i-own, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, to-read} -> {library, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, owned-books} -> {library, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, library} -> {owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{to-read} -> {books-i-own, library, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{owned-books} -> {books-i-own, library, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{library} -> {books-i-own, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own} -> {library, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{like, own

{owned} -> {favorites, library, like} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{library} -> {favorites, like, owned} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{favorites} -> {library, like, owned} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{library, like, owned-books} -> {favorites} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{favorites, like, owned-books} -> {library} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{favorites, library, owned-books} -> {like} (conf: 1.000, supp: 0.134, lift: 1.043, conv: 41237113.402)
{favorites, library, like} -> {owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{like, owned-books} -> {favorites, library} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{library, owned-books} -> {favorites, like} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{library, like} -> {favorites, owned-books} (conf: 1.000, supp: 0.134, lif

{currently-reading, owned} -> {books-i-own, favorites, library} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, library} -> {books-i-own, favorites, owned} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites} -> {books-i-own, library, owned} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, owned} -> {currently-reading, favorites, library} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, library} -> {currently-reading, favorites, owned} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, favorites} -> {currently-reading, library, owned} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, currently-reading} -> {favorites, library, owned} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{owned} -> {books-i-own, currently-reading, favorites, library} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)


{library, like, owned-books} -> {books-i-own, default} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{default, like, owned-books} -> {books-i-own, library} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{default, library, owned-books} -> {books-i-own, like} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{default, library, like} -> {books-i-own, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, like, owned-books} -> {default, library} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, library, owned-books} -> {default, like} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, library, like} -> {default, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, default, owned-books} -> {library, like} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, default, like} -> {library, owned-books} (conf: 1.000, sup

...

{books-i-own, favorites} -> {default, library, owned, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{books-i-own, default} -> {favorites, library, owned, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{to-read} -> {books-i-own, default, favorites, library, owned, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{owned-books} -> {books-i-own, default, favorites, library, owned, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{owned} -> {books-i-own, default, favorites, library, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{library} -> {books-i-own, default, favorites, owned, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{favorites} -> {books-i-own, default, library, owned, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{default} -> {books-i-own, favorites, lib

{currently-reading, owned-books} -> {default, favorites, like, owned, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, owned} -> {default, favorites, like, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, like} -> {default, favorites, owned, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites} -> {default, like, owned, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, default} -> {favorites, like, owned, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{to-read} -> {currently-reading, default, favorites, like, owned, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{owned-books} -> {currently-reading, default, favorites, like, owned, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{owned} -> {currentl

{currently-reading, library, owned-books, to-read} -> {books-i-own, default, favorites, like} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, library, like, to-read} -> {books-i-own, default, favorites, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, library, like, owned-books} -> {books-i-own, default, favorites, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites, owned-books, to-read} -> {books-i-own, default, library, like} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites, like, to-read} -> {books-i-own, default, library, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites, like, owned-books} -> {books-i-own, default, library, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites, library, to-read} -> {books-i-own, defa

{currently-reading, favorites, like, owned, to-read} -> {books-i-own, library, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites, like, owned, owned-books} -> {books-i-own, library, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites, library, owned-books, to-read} -> {books-i-own, like, owned} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites, library, owned, to-read} -> {books-i-own, like, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites, library, owned, owned-books} -> {books-i-own, like, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites, library, like, to-read} -> {books-i-own, owned, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{currently-reading, favorites, library, like, owned-books} -> {books-i-own, owned,

{favorites, owned} -> {currently-reading, default, library, like, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{favorites, like} -> {currently-reading, default, library, owned, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{favorites, library} -> {currently-reading, default, like, owned, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{default, to-read} -> {currently-reading, favorites, library, like, owned, owned-books} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{default, owned-books} -> {currently-reading, favorites, library, like, owned, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{default, owned} -> {currently-reading, favorites, library, like, owned-books, to-read} (conf: 1.000, supp: 0.134, lift: 7.462, conv: 865979381.443)
{default, like} -> {currently-reading, favorites, library, owned, owned-books, to-read} (conf: 1.000, supp

## Rating Prediction <a class="anchor" id="rating-prediction"></a>

Let’s say we are predicting ratings for user A. For each item i, we check if the attributes of i “fire” any of A’s rules. A rule is said to be fired by an item if the left-hand side of the rules is a subset the item’s attributes. We then sort this set of rules by descending confidence levels.

In [9]:
highRules = set(filter(lambda rule: 'like' in rule.rhs and len(rule.rhs) == 1, allRules))
lowRules = set(filter(lambda rule: 'dislike' in rule.rhs and len(rule.rhs) == 1, allRules))

In [10]:
total = 0
rules_fired = 0
predicted_ratings = []

# for book in all_tags_grouped['goodreads_book_id'].unique():
for book in all_tags_grouped['goodreads_book_id'].unique():
    # print(book)
    explanation = []
    tags = set(all_tags_grouped.get_group(book[0])['tag_name'].unique())
    # print(list(highRules))
    for rule in highRules:
        rule_words = set([key for key in rule.lhs])
        #print("flag: ", book)
        if rule_words.issubset(tags):
            
            explanation.extend([key for key in rule.lhs])
            total += 1
            
    for rule in lowRules:
        rule_words = set([key for key in rule.lhs])
        
        if rule_words.issubset(tags):
            explanation.extend([key for key in rule.lhs])
            total -= 1
    predicted_ratings.append([booksRaw[booksRaw["book_id"] == book[0]].title,explanation,total])
    total = 0
    #print("The predicted rating for book ", book[0], "is ", predicted_rating)

In [11]:
predicted_ratings = pd.DataFrame(predicted_ratings,columns=["book_name","explanation","predicted_rating"])
display(predicted_ratings.head())

,book_name,explanation,predicted_rating
0,26 Harry Potter and the Half-Blood Prince (...,"[currently-reading, default, library, currentl...",254
1,20 Harry Potter and the Order of the Phoeni...,"[currently-reading, default, library, currentl...",254
2,1 Harry Potter and the Sorcerer's Stone (Ha...,"[currently-reading, default, library, currentl...",254
3,17 Harry Potter and the Prisoner of Azkaban...,"[currently-reading, default, library, currentl...",254
4,23 Harry Potter and the Goblet of Fire (Har...,"[owned, owned-books, to-read, owned, owned-boo...",31


In conclusion, rule-based recommender systems are simple in concept, but flexible and highly interpretable. They can be used as a model for content-based system, but also for collaborative filtering (a rule might look like {Likes x} → {Dislikes y}).